In [51]:
import requests
from datetime import date
import os
import os.path
import json
import copy
from concurrent.futures import ThreadPoolExecutor, as_completed

In [54]:
date_to_str = date.today().isoformat()
url = f"https://clearcut.quantumobile.com/api/clearcuts/info/2019-04-01/{date_to_str}"
resp = requests.get(url=url)
data = resp.json()

In [55]:
plotly_layout = {
    "title": "Cutting area changes",
    "yaxis": {
        "title": "Area, m²"
    },
    "xaxis": {
        "type": "category",
        "tickangle": -45
    }
}
plotly_data_template = [
    {
        "x": [],
        "y": [],
        "type": "bar"
    }
]
colors = ["#ff394a", "#ffed57", "#2d4ab9"]

def enrich_feature(feature):
    new_properties = dict()
    new_properties["style"] = {
        "color": colors[feature["properties"]["color"]] if feature["properties"]["color"] < 3 else "#fff"
    }
    new_properties["layout"] = plotly_layout
    pk = feature["properties"]["pk"]
    area_chart_url = f"https://clearcut.quantumobile.com/api/clearcuts/area_chart/{pk}/2019-04-01/{date_to_str}"
    resp = requests.get(url=area_chart_url)
    area_chart_data = resp.json()
    plotly_data = copy.deepcopy(plotly_data_template)
    for x in area_chart_data:
        plotly_data[0]["x"].append(x["image_date_current"])
        plotly_data[0]["y"].append(x["zone_area"])
    new_properties["data"] = plotly_data
    feature["properties"] = new_properties

with ThreadPoolExecutor(max_workers=50) as executor:
    futures = [executor.submit(enrich_feature, x) for x in data["features"]]
    i = 0
    for future in as_completed(futures):
        i += 1
        if i % 1000 == 0:
            print(str(i) + "/" + str(len(data["features"])))

1000/56634
2000/56634
3000/56634
4000/56634
5000/56634
6000/56634
7000/56634
8000/56634
9000/56634
10000/56634
11000/56634
12000/56634
13000/56634
14000/56634
15000/56634
16000/56634
17000/56634
18000/56634
19000/56634
20000/56634
21000/56634
22000/56634
23000/56634
24000/56634
25000/56634
26000/56634
27000/56634
28000/56634
29000/56634
30000/56634
31000/56634
32000/56634
33000/56634
34000/56634
35000/56634
36000/56634
37000/56634
38000/56634
39000/56634
40000/56634
41000/56634
42000/56634
43000/56634
44000/56634
45000/56634
46000/56634
47000/56634
48000/56634
49000/56634
50000/56634
51000/56634
52000/56634
53000/56634
54000/56634
55000/56634
56000/56634


In [56]:
NB_USER = os.getenv('NB_USER')
temp_path = f"/home/{NB_USER}/work/results/clearcut/ClearCut-{date_to_str}"
os.makedirs(os.path.dirname(temp_path), exist_ok=True)
with open(temp_path, "w") as outfile:
    json.dump(data, outfile)
os.rename(temp_path, temp_path + ".geojson")